In [179]:
import keras as k
import keras.layers as l
from keras.models import load_model
import dataset_tools as dt

import random
import matplotlib.pyplot as plt
import numpy as np
import sys
import os

In [ ]:
dataset_1 = '/mnt/data/deepfake/datasets/faces/Actor_01/'
dataset_2 = '/mnt/data/deepfake/datasets/faces/Actor_02/'

pics_a = dt.get_pics(dataset_1)
pics_b = dt.get_pics(dataset_2)

In [193]:
class df_net:
    def __init__(self,pics_a, pics_b, ratio = 0.9):
        self.DIM = 100 #image dimension
        Xa, Xb = dt.format_images(pics_a, pics_b, self.DIM, self.DIM)
        self.train_a, self.test_a = dt.separate_dataset(Xa, ratio)
        self.train_b, self.test_b = dt.separate_dataset(Xb, ratio)
                
    def train(self, n_epochs = 1, verbose=0) :
        for n in range(n_epochs):
            print("Epoch %d / %d" % ((n+1),n_epochs), end='\r')
            self.ae_a.fit(self.train_a, self.train_a, epochs=1, verbose=verbose)
            self.ae_b.fit(self.train_b, self.train_b, epochs=1, verbose=verbose)
            
    def save(self, folder) :
        os.makedirs(folder, exist_ok=True)
        self.ae_a.save(folder + "/ae_a.h5")
        self.ae_b.save(folder + "/ae_b.h5")
        
    def load(self, folder) :
        self.ae_a = load_model(folder + "/ae_a.h5")
        self.ae_b = load_model(folder + "/ae_b.h5")

In [194]:
class dense_simple(df_net):
    def __init__(self,pics_a, pics_b, ratio = 0.9):
        df_net.__init__(self, pics_a, pics_b, ratio)
        self.create_network()
        
    def create_network(self) :
        DIM = self.DIM
        input_layer = l.Input(shape = (DIM * DIM,))
        encoded = l.Dense(32, activation='relu')(input_layer)
        decoded_a = l.Dense(DIM*DIM, activation = "sigmoid")(encoded)
        decoded_b = l.Dense(DIM*DIM, activation = "sigmoid")(encoded)
        self.ae_a = k.Model(input_layer, decoded_a)
        self.ae_b = k.Model(input_layer, decoded_b)
        encodeur = k.Model(input_layer, encoded)
        
        self.ae_a.compile(optimizer='adadelta', loss='binary_crossentropy')
        self.ae_b.compile(optimizer='adadelta', loss='binary_crossentropy')

In [200]:
class cnn_simple(df_net):
    def __init__(self,pics_a, pics_b, ratio = 0.9):
        df_net.__init__(self, pics_a, pics_b, ratio)
        
        self.train_a = self.train_a.reshape((len(self.train_a), 100,100,1))
        self.train_b = self.train_b.reshape((len(self.train_b), 100,100,1))
        self.test_a = self.test_a.reshape((len(test_xa), 100,100,1))
        test_xb = test_xb.reshape((len(test_xb), 100,100,1))
        self.create_network()
        
    def create_network(self) :
        input_img = l.Input(shape=(self.DIM, self.DIM, 1))  # adapt this if using `channels_first` image data format

        x = l.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
        x = l.MaxPooling2D((2, 2), padding='same')(x)
        x = l.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
        x = l.MaxPooling2D((2, 2), padding='same')(x)
        x = l.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
        encoded = l.MaxPooling2D((2, 2), padding='same')(x)

        # at this point the representation is (4, 4, 8) i.e. 128-dimensional

        xa = l.Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
        xa = l.UpSampling2D((2, 2))(xa)
        xa = l.Conv2D(8, (3, 3), activation='relu', padding='same')(xa)
        xa = l.UpSampling2D((2, 2))(xa)
        xa = l.Conv2D(16, (3, 3), activation='relu')(xa)
        xa = l.UpSampling2D((2, 2))(xa)
        decoded_a = l.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(xa)

        xb = l.Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
        xb = l.UpSampling2D((2, 2))(xb)
        xb = l.Conv2D(8, (3, 3), activation='relu', padding='same')(xb)
        xb = l.UpSampling2D((2, 2))(xb)
        xb = l.Conv2D(16, (3, 3), activation='relu')(xb)
        xb = l.UpSampling2D((2, 2))(xb)
        decoded_b = l.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(xb)

        self.ae_a = k.Model(input_img, decoded_a)
        self.ae_b = k.Model(input_img, decoded_b)

        encoder = k.Model(input_img, encoded)

        self.ae_a.compile(optimizer='adadelta', loss='binary_crossentropy')
        self.ae_b.compile(optimizer='adadelta', loss='binary_crossentropy')

In [201]:
testDense = cnn_simple(pics_a, pics_b)

testDense.train(5)

test_img = testDense.test_a[10]

plt.subplot(1,2,1)
plt.imshow(test_img.reshape(100,100), cmap='gray')
plt.subplot(1,2,2)
pred = testDense.ae_a.predict(np.array([test_img]))
plt.imshow(pred[0].reshape(100,100), cmap='gray')

ValueError: Error when checking input: expected input_58 to have 4 dimensions, but got array with shape (900, 10000)

In [197]:
testDense = dense_simple(pics_a, pics_b)
testDense.save('.')